# Boyish Scraping

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from random import randint
import time, urllib, json, unicodedata
from tqdm import tqdm

# Headers to imitate browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Mobile Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.5',
    'DNT': '1',
    'Referer': 'https://www.google.co.uk',
    'Upgrade-Insecure-Requests': '1'
}

C:\Users\linds\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
main_URL = 'https://www.boyish.com'
URL_extension_for_products = '/products.json?limit=250&page='
current_page_number = 1

all_URLs_and_JSON = {}
current_product_list = None

while current_product_list != []:
    # make the full json url
    url = main_URL + URL_extension_for_products + str(current_page_number)
    #print(url)
    
    # Get the page
    req = urllib.request.Request(url, headers=headers)
    with urllib.request.urlopen(req) as request:
        s = request.read()
        response = json.loads(s)
        current_product_list = response['products']
    
    time.sleep(randint(4,6))
    
    # construct the URL
    for product in current_product_list:
        product_url = main_URL + '/products/' + product['handle']
        all_URLs_and_JSON[product_url] = product
        
    current_page_number += 1

In [3]:
print(len(all_URLs_and_JSON))
#all_URLs_and_JSON

173


In [4]:
def find_product_name(soup, product_JSON):
    return product_JSON['title'].strip()

def find_material(soup, product_JSON):
    materials = {}
    materials_container = soup.find('div', class_ = 'Product__Tabs')
    materials_string = materials_container.find('button', class_ = 'Collapsible__Button Heading u-h6').text.strip()
    materials_string = materials_string[14:]
    material_list_cleaned = []
    
    if '/' in materials_string:
        material_list_cleaned = materials_string.split('/')
    else:
        the_materials_split = materials_string.split(' ')
        current_material = ""
        how_many_digits = 0
        for index,item in enumerate(the_materials_split):
            if item[0].isdigit() and how_many_digits == 0:
                current_material = current_material + ' ' + item
                how_many_digits += 1
            elif item[0].isdigit() and how_many_digits == 1:
                material_list_cleaned.append(current_material.strip())
                current_material = ""
                current_material = current_material + ' ' + item
            else:
                current_material = current_material + ' ' + item
                if index == len(the_materials_split) - 1:
                    material_list_cleaned.append(current_material.strip())
        
    materials_list = [item.strip().replace('%', '% &ND') for item in material_list_cleaned]
    materials_split = [item.split('&ND') for item in materials_list]
    for item in materials_split:
        materials[item[1].strip()] = item[0].strip()

    return materials

def find_color(soup, product_JSON):
    color_container = product_JSON['options'][0]
    if color_container['name'] == 'Color':
        color = color_container['values'][0].lower()
    else:
        color = 'not defined'
    return [color]

def find_price(soup, product_JSON):
    price_text = soup.find('span', class_ = 'ProductMeta__Price Price Text--subdued u-h4').text.strip() 
    if price_text[0] == '$':
        price_text = price_text[1:] 
    return ('USD', price_text)

def find_size(soup, product_JSON):
    return [i.find('input').get('value') for i in soup.find('ul', {'class': 'SizeSwatchList HorizontalList HorizontalList--spacingTight'}).find_all('li')]

def find_image(soup, product_JSON):
    return product_JSON['images'][0]['src']

def find_brand(soup, product_JSON):
    brand = product_JSON['vendor']
    return brand

def find_description(soup, product_JSON):
    description_container = soup.find('div', class_ = 'Rte')
    
    if description_container.text:
        description = [description_container.text.strip()]
    else:
        description_container.find_all('p')
        description = []
        for item in description_container:
            if item.find('br'):
                replaced_string = item.text.replace('\"', ' inches\"')
                split_lines = replaced_string.split('\"')
                for line in split_lines:
                    description.append(line.replace('\n', ' ').strip())
            else:
                description.append(item.text.replace('\n', ' ').strip())
    
    return description

In [5]:
def scrape_single_product(URL, html_soup, product_JSON):
    product = {}

    product['Name'] = find_product_name(html_soup, product_JSON)
    product['Material'] = find_material(html_soup, product_JSON)
    product['Color'] = find_color(html_soup, product_JSON)
    product['Price'] = find_price(html_soup, product_JSON)
    product['Size'] = find_size(html_soup, product_JSON)
    product['URL'] = URL
    product['Image'] = find_image(html_soup, product_JSON)
    product['Brand_name'] = find_brand(html_soup, product_JSON)
    product['Description'] = find_description(html_soup, product_JSON)
    return product

In [6]:
def is_available(product_JSON):
    variants = product_JSON['variants']
    
    is_available = False
    
    for variant in variants:
        availability = variant['available']
        if availability == True:
            is_available = True
    
    return is_available

In [7]:
# # These 3 cells scrape a single product
# URL_to_scrape = 'https://www.boyish.com/products/the-julian-like-crazy'
# page = requests.get(URL_to_scrape, headers = headers)
# single_soup = bs(page.content, 'html.parser')

# product_as_json = None
# URL_to_scrape_JSON = URL_to_scrape + ".json"

# single_req = urllib.request.Request(URL_to_scrape_JSON, headers=headers)
# with urllib.request.urlopen(single_req) as request:
#     s = request.read()
#     response = json.loads(s)
#     product_as_json = response['product']
#     print(product_as_json)

# singleProductResult  = [scrape_single_product(URL_to_scrape, single_soup, product_as_json)]
# print(singleProductResult)
# product_data_frame = pd.DataFrame(data = singleProductResult,)
# product_data_frame.to_csv('single_item.csv', encoding='utf-8-sig')
# pd.DataFrame(data = singleProductResult)

In [8]:
all_products = []
failed_urls = []
unavailable_products = []

# counter = 0

for url, the_JSON in tqdm(all_URLs_and_JSON.items()):
    try:
        #print(url)
        if is_available(the_JSON):
            page = requests.get(url, headers = headers)
            soup = bs(page.content, 'html.parser')

            product = scrape_single_product(url, soup, the_JSON)
            #print(product)
            all_products.append(product)
        else:
            unavailable_products.append(url)

        time.sleep(randint(5,15))
  
    except:
        failed_urls.append(url)
    
#     if counter == 15:
#         break
#     counter += 1

print('\nSummary\n')
print(len(failed_urls), 'Failed URLs: ', failed_urls)
print(len(unavailable_products), 'Unavailable Products: ', unavailable_products)
print('Total products collected: ', len(all_products))
print('Done!')

100%|██████████| 173/173 [27:00<00:00,  9.37s/it]


Summary

27 Failed URLs:  ['https://www.boyish.com/products/the-nori-black-beauty', 'https://www.boyish.com/products/the-kenny-my-fair-lady', 'https://www.boyish.com/products/the-kenny-modern-times', 'https://www.boyish.com/products/the-robby-golden-child', 'https://www.boyish.com/products/moss-top-topaz-biosculpt', 'https://www.boyish.com/products/the-robby-endless-summer', 'https://www.boyish.com/products/the-angel-endless-summer', 'https://www.boyish.com/products/california-high-leg-topaz-biosculpt', 'https://www.boyish.com/products/jenna-bodysuit-topaz-biosculpt', 'https://www.boyish.com/products/routeins', 'https://www.boyish.com/products/boyish-scarf-pony-light-blue-denim', 'https://www.boyish.com/products/boyish-face-mask-crystal-pink', 'https://www.boyish.com/products/boyish-face-mask-black', 'https://www.boyish.com/products/boyish-face-mask-white', 'https://www.boyish.com/products/guppyfriend-washing-bag-eco-clear', 'https://www.boyish.com/products/the-paul-juniper', 'https:/

In [ ]:
fail_again_URLs = []
for url in failed_urls:
    try:
        req = urllib.request.Request(url, headers=headers)
        
        with urllib.request.urlopen(req) as request:
            page = request.read()
            soup = bs(page, 'html.parser')
            all_products.append(scrape_single_product(url, soup))
            
            time.sleep(randint(5,15))
    except:
        fail_again_URLs.append(url)

In [ ]:
product_data_frame = pd.DataFrame(data = all_products)
product_data_frame.to_csv('boyish_table.csv', encoding='utf-8-sig')
pd.DataFrame(data = all_products)

,Name,Material,Color,Price,Size,URL,Image,Brand_name,Description
0,The Zachary | Mirror,"{'OCS Certified Organic Cotton': '66%', 'Tence...",[mirror],"(USD, 158)","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]",https://www.boyish.com/products/the-zachary-mi...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[A little give and take. The Zachary in Mirror...
1,The Tommy Stretch | Eternal Sunshine,"{'OCS Certified Organic Cotton': '17%', 'GRS C...",[eternal sunshine],"(USD, 168)","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]",https://www.boyish.com/products/the-tommy-stre...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,"[The classic straight jean you love, this time..."
2,The Ricky | Natural,"{'OCS Certified Organic Cotton': '69%', 'Tence...",[natural],"(USD, 158)","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]",https://www.boyish.com/products/the-ricky-natural,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[A little Boyish flare. The Ricky in Natural i...
3,The Monty | Ruby Sparks,"{'OCS Certified Organic Cotton': '17%', 'GRS C...",[ruby sparks],"(USD, 108)","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]",https://www.boyish.com/products/the-monty-ruby...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[Your new go-to Spring to Summer short. The Mo...
4,The Bradley | Sunrise,"{'OCS Certified Organic Cotton': '60%', 'Tence...",[sunrise],"(USD, 108)","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]",https://www.boyish.com/products/the-bradley-su...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[The ultimate (and longest) Boyish shorts. Our...
...,...,...,...,...,...,...,...,...,...
153,The Westley | Pandora's Box,"{'Organic Cotton': '70%', 'GRS Certified Recyc...",[pandora's box],"(USD, 188)","[XXS, XS, S, M, L, XL]",https://www.boyish.com/products/the-westley-pa...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[Ready for an adventure. The Westley is the pe...
154,The Andy | Senso,"{'OCS Certified Organic Cotton': '66%', 'Tence...",[senso],"(USD, 108)","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]",https://www.boyish.com/products/the-andy-senso,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[Perfectly imperfect. The Andy is a twist on a...
155,The Billy | La Strada,"{'OCS Certified Organic Cotton': '18%', 'GRS C...",[la strada],"(USD, 168)","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]",https://www.boyish.com/products/the-billy-la-s...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[Your new classic. The Billy is a high waisted...
156,The Charley | Greed,"{'OCS Certified Organic Cotton': '60%', 'Tence...",[greed],"(USD, 188)","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]",https://www.boyish.com/products/the-charley-greed,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[Back in time. Here is our take on rigid high ...
